In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
tf.random.set_seed(777)

In [9]:
def min_max_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7),np.max(data, 0),np.min(data, 0)
def min_max_scaler_val_test(data,MAX,MIN):
    numerator = data - MIN
    denominator = MAX - MIN
    return numerator / (denominator + 1e-7)

In [10]:
xy=pd.read_csv('winequality-red.csv')
xy.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


In [11]:
xy.tail(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1589,6.6,0.725,0.20,7.8,0.073,29.0,79.0,0.99770,3.29,0.54,9.2,5
1590,6.3,0.550,0.15,1.8,0.077,26.0,35.0,0.99314,3.32,0.82,11.6,6
1591,5.4,0.740,0.09,1.7,0.089,16.0,26.0,0.99402,3.67,0.56,11.6,6
1592,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1593,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
1598,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6


In [12]:
# xy.drop(xy.columns[[2,4,5,6,7,8,9]], axis='columns')
# xy.head(10)

In [13]:
Y_value = xy['quality'].unique()

for i in range(len(Y_value)):
    xy[(xy['quality'] == i+3)] = i 

xy=xy.values


In [15]:
#xy[:,:-1]=min_max_scaler(xy[:,:-1])
np.random.shuffle(xy)

val_line=int(len(xy)*0.6)
test_line=int(len(xy)*0.8)
x=xy[:,:-1]
y=xy[:,-1]
y_one_hot = tf.keras.utils.to_categorical(y, len(Y_value))
x_data=x[:val_line,:]
x_data[:,:],x_data_max,x_data_min=min_max_scaler(x_data[:,:])
y_data=y_one_hot[:val_line,:]
x_val=x[val_line:test_line,:]
x_val=min_max_scaler_val_test(x_val,x_data_max,x_data_min)
y_val=y_one_hot[val_line:test_line,:]
x_test=x[test_line:,:]
x_test=min_max_scaler_val_test(x_test,x_data_max,x_data_min)
y_test=y_one_hot[test_line:,:] 

x_reshape_data=np.reshape(x_data,[x_data.shape[0],x_data.shape[1],1])
x_reshape_val=np.reshape(x_val,[x_val.shape[0],x_val.shape[1],1])
x_reshape_test=np.reshape(x_test,[x_test.shape[0],x_test.shape[1],1])


In [18]:
drop_rate=0.3
tf.model=tf.keras.Sequential()
tf.model.add(tf.keras.layers.Conv1D(filters=40, kernel_size=6, input_shape=(x_reshape_data.shape[1],x_reshape_data.shape[2]),strides=1,activation='elu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))
# L2
tf.model.add(tf.keras.layers.Conv1D(filters=40, kernel_size=2,strides=1, activation='elu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))
# L fully connected
tf.model.add(tf.keras.layers.Flatten())
tf.model.add(tf.keras.layers.Dense(units=50, activation='elu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))
tf.model.add(tf.keras.layers.Dense(units=len(Y_value), activation='softmax'))


tf.model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(lr=0.001), metrics=['accuracy'])
tf.model.summary()



# drop_rate=0.3
# tf.model = tf.keras.Sequential() 
# tf.model.add(tf.keras.layers.Dense(input_dim=x_data.shape[1], units=100))  # use_bias is True, by default 
# tf.model.add(tf.keras.layers.Dropout(drop_rate))
# tf.model.add(tf.keras.layers.Dense(units=50, activation='elu'))
# tf.model.add(tf.keras.layers.Dropout(drop_rate))
# tf.model.add(tf.keras.layers.Dense(units=25, activation='elu'))
# tf.model.add(tf.keras.layers.Dropout(drop_rate))
# tf.model.add(tf.keras.layers.Dense(units=len(Y_value), activation='softmax'))
# # use loss == categorical_crossentropy 
# tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy']) 
# tf.model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 6, 40)             280       
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 40)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 5, 40)             3240      
_________________________________________________________________
dropout_4 (Dropout)          (None, 5, 40)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                10050     
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)               

In [19]:
history = tf.model.fit(x_reshape_data, y_data, epochs=500,validation_data=(x_reshape_val,y_val)) 
#prediction_test= tf.model.predict(x_test)

Epoch 1/500
30/30 [==============================] - 1s 7ms/step - loss: 1.5256 - accuracy: 0.3674 - val_loss: 1.1506 - val_accuracy: 0.8313
Epoch 2/500
30/30 [==============================] - 0s 2ms/step - loss: 1.2371 - accuracy: 0.4449 - val_loss: 1.1011 - val_accuracy: 0.8313
Epoch 3/500
30/30 [==============================] - 0s 2ms/step - loss: 1.1356 - accuracy: 0.5067 - val_loss: 0.9544 - val_accuracy: 0.8313
Epoch 4/500
30/30 [==============================] - 0s 2ms/step - loss: 0.9580 - accuracy: 0.7108 - val_loss: 0.6827 - val_accuracy: 0.8313
Epoch 5/500
30/30 [==============================] - 0s 2ms/step - loss: 0.6882 - accuracy: 0.8140 - val_loss: 0.4096 - val_accuracy: 0.8313
Epoch 6/500
30/30 [==============================] - 0s 2ms/step - loss: 0.4438 - accuracy: 0.8861 - val_loss: 0.2283 - val_accuracy: 0.9469
Epoch 7/500
30/30 [==============================] - 0s 2ms/step - loss: 0.3235 - accuracy: 0.9210 - val_loss: 0.1140 - val_accuracy: 0.9906
Epoch 8/500
3

Epoch 58/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 5.0663e-05 - val_accuracy: 1.0000
Epoch 59/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 2.1514e-05 - val_accuracy: 1.0000
Epoch 60/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0085 - accuracy: 0.9992 - val_loss: 3.9852e-05 - val_accuracy: 1.0000
Epoch 61/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0052 - accuracy: 0.9996 - val_loss: 2.4771e-04 - val_accuracy: 1.0000
Epoch 62/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0049 - accuracy: 0.9988 - val_loss: 3.1243e-05 - val_accuracy: 1.0000
Epoch 63/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 0.9982 - val_loss: 2.0454e-05 - val_accuracy: 1.0000
Epoch 64/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0189 - accuracy: 0.9925 - val_loss: 9.6678e-05 -

30/30 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.0040e-06 - val_accuracy: 1.0000
Epoch 114/500
30/30 [==============================] - 0s 2ms/step - loss: 5.3645e-04 - accuracy: 1.0000 - val_loss: 1.7531e-06 - val_accuracy: 1.0000
Epoch 115/500
30/30 [==============================] - 0s 2ms/step - loss: 5.6161e-04 - accuracy: 0.9997 - val_loss: 1.5951e-04 - val_accuracy: 1.0000
Epoch 116/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0170 - accuracy: 0.9919 - val_loss: 1.6391e-05 - val_accuracy: 1.0000
Epoch 117/500
30/30 [==============================] - 0s 2ms/step - loss: 4.9202e-04 - accuracy: 1.0000 - val_loss: 1.0669e-06 - val_accuracy: 1.0000
Epoch 118/500
30/30 [==============================] - 0s 2ms/step - loss: 2.2701e-04 - accuracy: 1.0000 - val_loss: 1.1194e-06 - val_accuracy: 1.0000
Epoch 119/500
30/30 [==============================] - 0s 2ms/step - loss: 1.6256e-04 - accuracy: 1.0000 - val_loss:

Epoch 168/500
30/30 [==============================] - 0s 2ms/step - loss: 1.8354e-04 - accuracy: 1.0000 - val_loss: 8.6427e-08 - val_accuracy: 1.0000
Epoch 169/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 7.1153e-08 - val_accuracy: 1.0000
Epoch 170/500
30/30 [==============================] - 0s 2ms/step - loss: 1.1856e-04 - accuracy: 1.0000 - val_loss: 8.5682e-09 - val_accuracy: 1.0000
Epoch 171/500
30/30 [==============================] - 0s 2ms/step - loss: 6.3813e-05 - accuracy: 1.0000 - val_loss: 7.4506e-09 - val_accuracy: 1.0000
Epoch 172/500
30/30 [==============================] - 0s 2ms/step - loss: 1.1200e-04 - accuracy: 1.0000 - val_loss: 1.8254e-08 - val_accuracy: 1.0000
Epoch 173/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 0.9993 - val_loss: 3.2894e-07 - val_accuracy: 1.0000
Epoch 174/500
30/30 [==============================] - 0s 2ms/step - loss: 2.4803e-04 - accuracy: 1.00

30/30 [==============================] - 0s 2ms/step - loss: 4.7663e-04 - accuracy: 1.0000 - val_loss: 4.1202e-07 - val_accuracy: 1.0000
Epoch 223/500
30/30 [==============================] - 0s 2ms/step - loss: 1.9088e-04 - accuracy: 1.0000 - val_loss: 2.3991e-07 - val_accuracy: 1.0000
Epoch 224/500
30/30 [==============================] - 0s 2ms/step - loss: 3.7068e-04 - accuracy: 1.0000 - val_loss: 2.5257e-07 - val_accuracy: 1.0000
Epoch 225/500
30/30 [==============================] - 0s 2ms/step - loss: 7.5904e-05 - accuracy: 1.0000 - val_loss: 2.6561e-07 - val_accuracy: 1.0000
Epoch 226/500
30/30 [==============================] - 0s 2ms/step - loss: 6.9367e-05 - accuracy: 1.0000 - val_loss: 2.0005e-07 - val_accuracy: 1.0000
Epoch 227/500
30/30 [==============================] - 0s 2ms/step - loss: 4.0183e-05 - accuracy: 1.0000 - val_loss: 1.8664e-07 - val_accuracy: 1.0000
Epoch 228/500
30/30 [==============================] - 0s 2ms/step - loss: 5.4094e-04 - accuracy: 1.0000 - v

30/30 [==============================] - 0s 2ms/step - loss: 1.4252e-04 - accuracy: 1.0000 - val_loss: 1.3784e-08 - val_accuracy: 1.0000
Epoch 277/500
30/30 [==============================] - 0s 2ms/step - loss: 3.7570e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 278/500
30/30 [==============================] - 0s 2ms/step - loss: 3.7098e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 279/500
30/30 [==============================] - 0s 2ms/step - loss: 1.7896e-04 - accuracy: 1.0000 - val_loss: 3.7253e-10 - val_accuracy: 1.0000
Epoch 280/500
30/30 [==============================] - 0s 2ms/step - loss: 6.8875e-05 - accuracy: 1.0000 - val_loss: 6.3330e-09 - val_accuracy: 1.0000
Epoch 281/500
30/30 [==============================] - 0s 2ms/step - loss: 1.3178e-04 - accuracy: 1.0000 - val_loss: 3.7253e-10 - val_accuracy: 1.0000
Epoch 282/500
30/30 [==============================] - 0s 2ms/step - loss: 1.0734e-04 - accuracy: 1.0000 - v

Epoch 330/500
30/30 [==============================] - 0s 2ms/step - loss: 6.8723e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 331/500
30/30 [==============================] - 0s 2ms/step - loss: 7.1196e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 332/500
30/30 [==============================] - 0s 2ms/step - loss: 4.6271e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 333/500
30/30 [==============================] - 0s 2ms/step - loss: 5.0123e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 334/500
30/30 [==============================] - 0s 2ms/step - loss: 3.2690e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 335/500
30/30 [==============================] - 0s 2ms/step - loss: 9.8957e-04 - accuracy: 0.9997 - val_loss: 5.2154e-08 - val_accuracy: 1.0000
Epoch 336/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 

30/30 [==============================] - 0s 2ms/step - loss: 6.5312e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 385/500
30/30 [==============================] - 0s 2ms/step - loss: 4.2815e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 386/500
30/30 [==============================] - 0s 2ms/step - loss: 1.6655e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 387/500
30/30 [==============================] - 0s 2ms/step - loss: 5.1441e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 388/500
30/30 [==============================] - 0s 2ms/step - loss: 5.4392e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 389/500
30/30 [==============================] - 0s 2ms/step - loss: 1.0602e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 390/500
30/30 [==============================] - 0s 2ms/step - loss: 2.8449e-04 - accuracy: 1.0000 - v

30/30 [==============================] - 0s 2ms/step - loss: 0.0063 - accuracy: 0.9975 - val_loss: 4.8615e-07 - val_accuracy: 1.0000
Epoch 439/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0110 - accuracy: 0.9961 - val_loss: 2.9802e-09 - val_accuracy: 1.0000
Epoch 440/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0029 - accuracy: 0.9975 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 441/500
30/30 [==============================] - 0s 2ms/step - loss: 0.0032 - accuracy: 0.9993 - val_loss: 1.0431e-07 - val_accuracy: 1.0000
Epoch 442/500
30/30 [==============================] - 0s 2ms/step - loss: 2.1612e-04 - accuracy: 1.0000 - val_loss: 1.0431e-07 - val_accuracy: 1.0000
Epoch 443/500
30/30 [==============================] - 0s 2ms/step - loss: 5.0453e-05 - accuracy: 1.0000 - val_loss: 5.2154e-08 - val_accuracy: 1.0000
Epoch 444/500
30/30 [==============================] - 0s 2ms/step - loss: 8.3726e-05 - accuracy: 1.0000 - val_loss: 0.0000e

Epoch 493/500
30/30 [==============================] - 0s 2ms/step - loss: 1.2474e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 494/500
30/30 [==============================] - 0s 2ms/step - loss: 5.8390e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 495/500
30/30 [==============================] - 0s 2ms/step - loss: 1.2697e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 496/500
30/30 [==============================] - 0s 2ms/step - loss: 1.5690e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 497/500
30/30 [==============================] - 0s 2ms/step - loss: 2.5273e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 498/500
30/30 [==============================] - 0s 2ms/step - loss: 3.0786e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 499/500
30/30 [==============================] - 0s 2ms/step - loss: 2.5687e-06 - accura

In [ ]:
cost_val=tf.model.evaluate(x_reshape_test,y_test)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show